In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64

from io import BytesIO
from PIL import Image

import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# Loading the trained model
model = load_model('model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 36, 16)        4624      
_________________________________________________________________
activation_3 (Activation)    (None, 26, 36, 16)        0         
__________

In [4]:
#  Creating a web server that receives queries from the simulator
sio = socketio.Server()
app = Flask(__name__)


#  A function that sends control to the simulator
def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [ ]:
#  Image processing function
def process_image(img):
    return img[10:130:2, ::4, :]    #  image range

#  A function that receives data from the simulator
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])   #  current vehicle speed from the simulator
        image_str = data["image"]      #  current camera image from the simulator
        
        
        decoded = base64.b64decode(image_str)    #  Image decoding
        image = Image.open(BytesIO(decoded))     #  Convert to image
        image_array = np.asarray(image)          #  Convert image to matrix
        
        #  Reducing the image so that the neural network can process it
        img = process_image(image_array)      
        #  Adding the 4th dimension at the front, one image in batch
        img_batch = np.expand_dims(img, axis=0)           
        #  Steering angle prediction converted to float from img_batch
        steering_angle = float(model.predict(img_batch))   

        #  Position of the accelerator pedal (values between 0.0 and 1.0), 0 = stop, 0.1 slow, 1.0 fast
        throttle = 1.0         

        if speed < 10:         #   If the speed is < 10, press the accelerator pedal more
            throttle = 0.6     
        if speed > 15:         #   If the speed is > 15, start braking the car
            throttle = -0.1       
        
        #  Sending car control commands
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
       
    
#  Starting the server and blocking the cell with the code   
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(5472) wsgi starting up on http://0.0.0.0:4567
(5472) accepted ('127.0.0.1', 49335)
